In [205]:
import pandas as pd
import io
import uuid

In [180]:
df = pd.read_csv('/Users/carboni/Downloads/fuzzy_addresses.csv', dtype ='str')

In [181]:
def add_fuzzy_rows_corrected(sorted_df):
    fuzzy_rows = []
    for i in range(len(sorted_df) - 1):
        current_row = sorted_df.iloc[i]
        next_row = sorted_df.iloc[i + 1]

        # Only add fuzzy rows if the artist is the same in the next row
        if current_row['artist'] == next_row['artist']:
            # Compute the difference in days
            day_diff = (next_row['begin'] - current_row['end']).days

            # Check conditions for adding fuzzy rows
            if day_diff > 1:
                # If 'labeladdressArtist' is different, add two new rows
                if current_row['labeladdressArtist'] != next_row['labeladdressArtist']:
                    new_row1 = current_row.copy()
                    new_row1['begin'] = current_row['end']
                    new_row1['end'] = next_row['begin']
                    new_row1['Fuzzy'] = True

                    new_row2 = next_row.copy()
                    new_row2['begin'] = current_row['end']
                    new_row2['Fuzzy'] = True

                    fuzzy_rows.extend([new_row1, new_row2])
                # If 'labeladdressArtist' is the same, add one new row
                elif current_row['labeladdressArtist'] == next_row['labeladdressArtist']:
                    new_row = current_row.copy()
                    new_row['begin'] = current_row['end']
                    new_row['end'] = next_row['begin']
                    new_row['Fuzzy'] = True
                    fuzzy_rows.append(new_row)

    # Append fuzzy rows to the DataFrame and sort it again
    extended_df = pd.concat([sorted_df, pd.DataFrame(fuzzy_rows)], ignore_index=True)
    extended_df = extended_df.sort_values(by=['artist', 'begin'])
    return extended_df

In [182]:
df['begin'] = pd.to_datetime(df['begin'])

In [183]:
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str)
        return True
    except:
        return False

In [184]:
invalid_end_dates = df[~df['end'].apply(is_valid_date)]['end']
print("Invalid 'end' dates:")
print(invalid_end_dates)

Invalid 'end' dates:
17       0000-12-31T23:59:59
49       0000-12-31T23:59:59
50       0000-12-31T23:59:59
63       0105-12-31T23:59:59
102      0105-12-31T23:59:59
                ...         
20390    0105-12-31T23:59:59
20392    0105-12-31T23:59:59
20393    0105-12-31T23:59:59
20474    0000-12-31T23:59:59
20482    0000-12-31T23:59:59
Name: end, Length: 1254, dtype: object


In [185]:
df = df[df['end'].apply(is_valid_date)]

In [186]:
df['end'] = pd.to_datetime(df['end'])

In [187]:
# Sort the DataFrame by 'begin'
df_sorted = df.sort_values(by=['artist', 'begin'])

In [188]:
df_sorted.head(10)

,artist,labelartist,addressArtist,city,labeladdressArtist,addressWKT,timespan,begin,end
6451,https://artlas.huma-num.fr/basart/resources/ag...,Eva Alexandre,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place d'Aine 9,POINT(1.254671 45.829867),https://artlas.huma-num.fr/basart/resources/ex...,1908-01-01,1908-12-31 23:59:59
6108,https://artlas.huma-num.fr/basart/resources/ag...,Adèle Kleinmann,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Caulaincourt 11,POINT(2.3326649 48.8869738),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01,1903-12-31 23:59:59
6107,https://artlas.huma-num.fr/basart/resources/ag...,Adèle Kleinmann,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,boulevard de Clichy 130 bis,POINT(2.3376988 48.8824888),https://artlas.huma-num.fr/basart/resources/ex...,1904-01-01,1904-12-31 23:59:59
18259,https://artlas.huma-num.fr/basart/resources/ag...,Henry Gazan,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place Dancourt 10,POINT(2.3426931 48.8833811),https://artlas.huma-num.fr/basart/resources/ex...,1919-01-01,1919-12-31 23:59:59
17130,https://artlas.huma-num.fr/basart/resources/ag...,Camille Schmitz-Reycend,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Théophile-Gautier 15 bis,POINT(2.2742612 48.85052959999999),https://artlas.huma-num.fr/basart/resources/ex...,1919-01-01,1919-12-31 23:59:59
744,https://artlas.huma-num.fr/basart/resources/ag...,Claude Monet,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Vintimille 2,POINT(2.3284023 48.881781),https://artlas.huma-num.fr/basart/resources/ex...,1879-01-01,1879-12-31 23:59:59
15189,https://artlas.huma-num.fr/basart/resources/ag...,Henri BOISGONTIER,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Clotaire 3,POINT(2.3447133 48.8456665),https://artlas.huma-num.fr/basart/resources/ex...,1906-01-01,1906-12-31 23:59:59
19862,https://artlas.huma-num.fr/basart/resources/ag...,Léonard Baunattian,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place de la Gare,POINT(2.220076 48.814942),https://artlas.huma-num.fr/basart/resources/ex...,1924-01-01,1924-12-31 23:59:59
15272,https://artlas.huma-num.fr/basart/resources/ag...,Gaston Maximilien Desportes de Linières,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,avenue des Ternes 30 Pereire Sud,POINT(2.2945681 48.8787367),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01,1903-12-31 23:59:59
17991,https://artlas.huma-num.fr/basart/resources/ag...,Félicien Garcia,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Amelot 2 Bastille,POINT(2.3694803 48.8552718),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01,1903-12-31 23:59:59


In [189]:
# Add fuzzy rows
df_with_fuzzy_updated = add_fuzzy_rows_corrected(df_sorted)

In [190]:
df_with_fuzzy_updated.head(20)

,artist,labelartist,addressArtist,city,labeladdressArtist,addressWKT,timespan,begin,end,Fuzzy
0,https://artlas.huma-num.fr/basart/resources/ag...,Eva Alexandre,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place d'Aine 9,POINT(1.254671 45.829867),https://artlas.huma-num.fr/basart/resources/ex...,1908-01-01 00:00:00,1908-12-31 23:59:59,NaN
1,https://artlas.huma-num.fr/basart/resources/ag...,Adèle Kleinmann,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Caulaincourt 11,POINT(2.3326649 48.8869738),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01 00:00:00,1903-12-31 23:59:59,NaN
2,https://artlas.huma-num.fr/basart/resources/ag...,Adèle Kleinmann,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,boulevard de Clichy 130 bis,POINT(2.3376988 48.8824888),https://artlas.huma-num.fr/basart/resources/ex...,1904-01-01 00:00:00,1904-12-31 23:59:59,NaN
3,https://artlas.huma-num.fr/basart/resources/ag...,Henry Gazan,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place Dancourt 10,POINT(2.3426931 48.8833811),https://artlas.huma-num.fr/basart/resources/ex...,1919-01-01 00:00:00,1919-12-31 23:59:59,NaN
4,https://artlas.huma-num.fr/basart/resources/ag...,Camille Schmitz-Reycend,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Théophile-Gautier 15 bis,POINT(2.2742612 48.85052959999999),https://artlas.huma-num.fr/basart/resources/ex...,1919-01-01 00:00:00,1919-12-31 23:59:59,NaN
5,https://artlas.huma-num.fr/basart/resources/ag...,Claude Monet,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Vintimille 2,POINT(2.3284023 48.881781),https://artlas.huma-num.fr/basart/resources/ex...,1879-01-01 00:00:00,1879-12-31 23:59:59,NaN
6,https://artlas.huma-num.fr/basart/resources/ag...,Henri BOISGONTIER,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Clotaire 3,POINT(2.3447133 48.8456665),https://artlas.huma-num.fr/basart/resources/ex...,1906-01-01 00:00:00,1906-12-31 23:59:59,NaN
7,https://artlas.huma-num.fr/basart/resources/ag...,Léonard Baunattian,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,place de la Gare,POINT(2.220076 48.814942),https://artlas.huma-num.fr/basart/resources/ex...,1924-01-01 00:00:00,1924-12-31 23:59:59,NaN
8,https://artlas.huma-num.fr/basart/resources/ag...,Gaston Maximilien Desportes de Linières,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,avenue des Ternes 30 Pereire Sud,POINT(2.2945681 48.8787367),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01 00:00:00,1903-12-31 23:59:59,NaN
9,https://artlas.huma-num.fr/basart/resources/ag...,Félicien Garcia,https://artlas.huma-num.fr/basart/resources/ad...,https://artlas.huma-num.fr/basart/resources/ci...,rue Amelot 2 Bastille,POINT(2.3694803 48.8552718),https://artlas.huma-num.fr/basart/resources/ex...,1903-01-01 00:00:00,1903-12-31 23:59:59,NaN


In [191]:
df_with_fuzzy_updated.to_csv('/Users/carboni/Downloads/fuzzy_results.csv', index=False)

In [192]:
df_filtered = df_with_fuzzy_updated[df_with_fuzzy_updated['Fuzzy'] == True].copy()

In [193]:
df_filtered.loc[:, 'artist'] = df_filtered['artist'].str.replace("https://artlas.huma-num.fr/basart/resources/agent/", "", regex=False)

In [194]:
df_filtered.loc[:, 'addressArtist'] = df_filtered['addressArtist'].str.replace("https://artlas.huma-num.fr/basart/resources/address/", "", regex=False)

In [195]:
df_filtered.loc[:, 'timespan'] = df_filtered['timespan'].str.replace("https://artlas.huma-num.fr/basart/resources/exhibition/", "", regex=False)

In [196]:
df_filtered.loc[:, 'timespan'] = df_filtered['timespan'].str.replace("/time-span", "", regex=False)

In [197]:
df_filtered.loc[:, 'city'] = df_filtered['city'].str.replace("https://artlas.huma-num.fr/basart/resources/city/", "", regex=False)

In [198]:
df_filtered['begin'] = df_filtered['begin'].dt.strftime('%Y-%m-%dT%H:%M:%S')
df_filtered['end'] = df_filtered['end'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [203]:
def generate_uuid(name):
    name_str = str(name)
    return str(uuid.uuid3(uuid.NAMESPACE_DNS, name_str))

In [206]:
df_filtered['uuid_time'] = df_filtered['timespan'].apply(generate_uuid)

In [207]:
df_filtered.head()

,artist,labelartist,addressArtist,city,labeladdressArtist,addressWKT,timespan,begin,end,Fuzzy,uuid_time
19283,006dbe8d-484f-32e3-880d-696005e8acd8,Félix VALLOTTON,7180,7061,rue des Belles-Feuilles 59,POINT(2.2800508 48.8687523),18602,1906-12-31T23:59:59,1909-01-01T00:00:00,True,d3af37bc-2804-3343-b996-65c9f5b5cfc4
19284,008d9227-818a-3c62-ad56-23e4c06c8ac3,Albert CRETTÉ,9840,7061,rue d'Alésia 199,POINT(2.3149951 48.8314926),18617,1928-12-31T23:59:59,1934-01-01T00:00:00,True,8ecc7663-fc9f-35ef-bf9a-5a91b19d67ca
19285,009fb1a0-2637-33d3-a0cb-7dac983790bc,Paul-Emile Colin,13482,42383,boulevard de la République 48,POINT(2.463657 48.8939354),2407,1908-12-31T23:59:59,1919-01-01T00:00:00,True,d35ebde5-f469-380f-bccc-9441f3a7dd61
19286,009fb1a0-2637-33d3-a0cb-7dac983790bc,Paul-Emile Colin,7160,109712,Chemin latéral 24,POINT(2.323704 48.779533),599,1908-12-31T23:59:59,1919-12-31T23:59:59,True,23fb2b76-d88f-332f-8db1-7f646a9d874b
19287,009fb1a0-2637-33d3-a0cb-7dac983790bc,Paul-Emile Colin,5530,109712,chemin Latéral 24,POINT(2.323704 48.779533),599,1919-12-31T23:59:59,1921-01-01T00:00:00,True,23fb2b76-d88f-332f-8db1-7f646a9d874b


In [208]:
columns = df_filtered.columns

# Iterate through the columns
for col in columns:
    # Check if the column name contains a space
    if ' ' in col:
        # Replace spaces with underscores
        df_filtered.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [209]:
df_filtered = df_filtered.replace(r"^ +| +$", r"", regex=True)

In [210]:
df_filtered.to_xml('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/fuzzy/fuzzy_address.xml', root_name="data", pretty_print=True, index=False) 